In [5]:
import sys
import os
import pandas as pd
import numpy as np
import copy
import math
import matplotlib.pyplot as plt
import json
import utils
import newEstimator as NE
main_dir = "C:/Users/nourisa/Downloads/testProjs/omics"
sys.path.insert(0,main_dir)
specs = dict(
    o_df_dir = os.path.join(main_dir,'data','original_omics.xlsx'),
    df_dir = os.path.join(main_dir,'data','omics.csv'),
    time = [1,2,3,4,7,8,9,10,11,14,21],
    p_ID = 'Entry',  # The column name in the original data to be used as protein ID
    c_tag = 'ctr_',
    s_tag = 'mg_',
    c_func = lambda t: 'ctr_' + str(t),  # The tag used in the tailored dataframe for control
    s_func = lambda t: 'mg_' + str(t),  # The tag used in the tailored dataframe for sample
    o_c_func = lambda t: 'LogLFQ intensity ' + str(t) + '_0',  # Func to extract the control data from the original database 
    o_s_func = lambda t: 'LogLFQ intensity ' + str(t) + '_1',  # Func to extract the sample data from the original database
    ee = 0.5,  # Threshold for log2FC to retain the data
    min_s = 2,  # Min occurance in time series in order to retain the data  
)
## read the original data
o_df = pd.read_excel(specs['o_df_dir'])

In [2]:
# process the data
o_df_m = utils.rename_missing_symbols(o_df,**specs); 
df = utils.tailor_names(o_df_m, **specs)
df = utils.remove_zeros(df, **specs)
# extract sig proteins
df_sig = utils.sig_df(df, **specs)
print('Sig proteins: {}'.format(len(df_sig)))
# impute missing values: available techniques to try: interpolate, univariate and multivariate: https://scikit-learn.org/stable/modules/impute.html
df_interp = df_sig.interpolate() 
df_interp = utils.listwise_deletion(df_interp)
print('Sig interpolated proteins: {}'.format(len(df_interp)))

Remaining missing names:  []
Data size, original: 2562


C:\Users\nourisa\Downloads\testProjs\omics\utils.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['index'],axis=1,inplace = True)


Data size, rows with all zeros were removed: 2229
Sig proteins:  34
Sig proteins: 34
Sig interpolated proteins: 34


In [ ]:
# stats of sig proteins: different time points, unknown proteins

# number of sig proteins for different time points
sig_prot_names = {} 
for day in specs['time']:
    sig_prot_names[day] = utils.sig_test(df,day,**specs)[specs['p_ID']].tolist()
# plot number of sig proteins for measurement times 
fig, axes = plt.subplots(figsize = (15,4), nrows = 1, ncols = 3, tight_layout = True)
def plot_stats_time_1(ax):
    ax.bar(range(len(sig_prot_names)),[len(x) for x in sig_prot_names.values()])
    ax.set_ylabel('Number of proteins')
    ax.set_xlabel('Measurement day')
    ax.set_xticks(range(len(sig_prot_names)))
    _ = ax.set_xticklabels(sig_prot_names.keys())
    ax.set_title('Differentially expressed proteins')
plot_stats_time(axes[0])
#------
# sig incidence number: how many times a protein is detected sig across different time points
limit_n = 3 # if the incidences are below this, dont plot it
sig_prots_e = [x for xx in sig_prot_names.values() for x in xx] # duplicates included
sig_prots = list(set(sig_prots_e)) # duplicates excluded
print('Number of prots upregulated in at least one time point: ',len(sig_prots))
sig_prots_i = {i:sig_prots_e.count(i) for i in sig_prots} # incidences 
sig_prots_i_f = {}
for key,value in sig_prots_i.items():
    if value >= limit_n:
        sig_prots_i_f[key] = value
print('Number of prots upregulated in at least 3 time points: ',len(sig_prots_i_f))
def plot_stats_time_2(ax):
    ax.bar(range(len(sig_prots_i_f)),sig_prots_i_f.values())
    ax.set_ylabel('Number of incidences')
    ax.set_xlabel('Measurement day')
    ax.set_xticks(range(len(sig_prots_i_f)))
    ax.set_xticklabels(sig_prots_i_f.keys())
    ax.set_title('Multiple differentially expressed proteins')
plot_stats_time_2(axes[1])
#----
# sig incidence number for unknown params
unknown_sigs_prots = [x for x in sig_prots if 'p_' in x ]
unknown_sigs_prots_i = {i:sig_prots_e.count(i) for i in unknown_sigs_prots} # incidences 
print('Total number of unknown paramters: ',len([x for x in df[specs['p_ID']].tolist() if 'p_' in x ]))
print('Sig number of unknown paramters: ',len(unknown_sigs_prots))
def plot_stats_time_3(ax):
    ax.bar(range(len(unknown_sigs_prots_i)),unknown_sigs_prots_i.values())
    ax.set_ylabel('Number of incidences')
    ax.set_xticks(range(len(unknown_sigs_prots_i)))
    _ = ax.set_xticklabels(unknown_sigs_prots_i.keys())
    ax.set_title('Multiple differentially expressed proteins')
plot_stats_time_3(axes[2])
#----
# output the name of sig proteins 
def remove_unknown(sig_prot_names):
    sig_prots_time_f = {}
    for key,values in sig_prot_names.items():
        values_f = [x for x in values if x != np.nan]
        _str = ''
        for value in values_f:
            _str += value + '\n'
        sig_prots_time_f[key] = _str
    return sig_prots_time_f
sig_prots_time_f = remove_unknown(sig_prot_names)
with open('results/sig_prots.txt','w') as f:
    for key,value in sig_prots_time_f.items():
        f.write('\n \nday '+ str(key)+': \n'+ value)

In [4]:
# reformat the data for time series
df_target = df_interp
time = specs['time']
p_ID = specs['p_ID']
data_ctr = np.array(df_target.loc[:,['ctr_'+str(day) for day in time]].values)
data_mg = np.array(df_target.loc[:,['mg_'+str(day) for day in time]].values)
(TS_data, time_points, decay_rates, gene_names) = \
    [data_ctr.transpose(),data_mg.transpose()], [time,time],[],df_target[p_ID].tolist()
print('n_exp n:',len(TS_data))
print('n_time*n_genes',TS_data[0].shape)

# convert data to n_samples*n_genes format
Xs,ys = NE.process_data(TS_data, time_points, regulators = 'all',alpha='from_data')

n_exp n: 2
n_time*n_genes (11, 34)


In [ ]:
## grid search
import newEstimator 
import importlib
import numpy as np
importlib.reload(newEstimator)
param = dict(
    estimator_t = 'RF', 
    n_estimators = 100,
    oob_score = True
)
param_grid = dict(
    max_depth = [10,30],
    n_estimators = [20,50,150]
)
specs = dict(
    n_jobs = 10,
#     cv = 5,
    use_oob_score = True,
)
best_scores, best_params, best_ests = newEstimator.grid_search(Xs, ys, param, param_grid, **specs)

In [ ]:
# network inference after training
import importlib
import newEstimator
import utils as ul
importlib.reload(utils)

ests, scores_train, links = newEstimator.network_inference(Xs, ys, ests = best_ests)
ul.plot_hist(xs = [scores_train,best_scores], names = ['train', 'test'])
print('train -> mean: %.3f, std: %.3f'%(np.mean(scores_train),np.std(scores_train)))
print('test -> mean: %.3f, std: %.3f'%(np.mean(best_scores),np.std(best_scores)))

In [ ]:
# network inference with no training
import importlib
import newEstimator
import utils as ul
importlib.reload(newEstimator)
param = dict(
    estimator_t = 'RF', 
    n_estimators = 100,
    oob_score = True
)
ests, scores_train, links, oob_scores = newEstimator.network_inference(Xs, ys, param = param)
ul.plot_hist(xs = [scores_train,best_scores], names = ['train', 'test'])
print('train -> mean: %.3f, std: %.3f'%(np.mean(scores_train),np.std(scores_train)))
print('test -> mean: %.3f, std: %.3f'%(np.mean(oob_scores),np.std(oob_scores)))